<a href="https://colab.research.google.com/github/Lokesh-Project/Time-Series-Forecasting/blob/main/TimeSeriesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib
import zipfile
import pandas as pd
import tensorflow as tf


# This function downloads and extracts the dataset to the directory that contains this file.

In [2]:
def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()

# This function normalizes the dataset using min max scaling.

In [3]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

# This function is used to map the time series dataset into windows of features and respective targets, to prepare it for training and validation.

In [4]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

# This function loads the data from CSV file, normalizes the data and splits the dataset into train and validation data. It also uses windowed_dataset() to split the data into windows of observations and targets. Finally, it defines, compiles and trains a neural network. This function returns the final trained model.

In [5]:
def solution_model():
    # Downloads and extracts the dataset to the directory that
    # contains this file.
    download_and_extract_data()
    # Reads the dataset from the CSV.
    df = pd.read_csv('household_power_consumption.csv', sep=',', infer_datetime_format=True, index_col='datetime', header=0)

    # Number of features in the dataset. We use all features as predictors to
    # predict all features at future time steps.
    N_FEATURES = len(df.columns)

    # Normalizes the data
    data = df.values
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))

    # Splits the data into training and validation sets.
    SPLIT_TIME = int(len(data) * 0.5)
    x_train = data[:SPLIT_TIME]
    x_valid = data[SPLIT_TIME:]

    # Clear any previous models from memory.
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

    # DO NOT CHANGE BATCH_SIZE IF YOU ARE USING STATEFUL LSTM/RNN/GRU.
    BATCH_SIZE = 32

    # Number of past time steps based on which future observations should be predicted.
    N_PAST = 24

    # Number of future time steps which are to be predicted.
    N_FUTURE = 24

    # By how many positions the window slides to create a new window of observations.
    SHIFT = 1

    # Code to create windowed train and validation datasets.
    train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE, shift=SHIFT)
    valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE, shift=SHIFT)

    # Code to define the model.
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(N_PAST, N_FEATURES)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.RepeatVector(N_FUTURE),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(N_FEATURES))
    ])

    # Code to compile the model
    model.compile(optimizer='adam', loss='mse')

    # Code to train the model
    model.fit(train_set, validation_data=valid_set, epochs=10)

    return model

In [6]:
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")

<ipython-input-5-85355796102e>:6: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('household_power_consumption.csv', sep=',', infer_datetime_format=True, index_col='datetime', header=0)


Epoch 1/10
1349/1349 [==============================] - 210s 144ms/step - loss: 0.0188 - val_loss: 0.0158
Epoch 2/10
1349/1349 [==============================] - 190s 141ms/step - loss: 0.0148 - val_loss: 0.0138
Epoch 3/10
1349/1349 [==============================] - 189s 140ms/step - loss: 0.0137 - val_loss: 0.0135
Epoch 4/10
1349/1349 [==============================] - 229s 170ms/step - loss: 0.0133 - val_loss: 0.0126
Epoch 5/10
1349/1349 [==============================] - 190s 141ms/step - loss: 0.0128 - val_loss: 0.0121
Epoch 6/10
1349/1349 [==============================] - 189s 140ms/step - loss: 0.0125 - val_loss: 0.0119
Epoch 7/10
1349/1349 [==============================] - 190s 141ms/step - loss: 0.0123 - val_loss: 0.0119
Epoch 8/10
1349/1349 [==============================] - 188s 139ms/step - loss: 0.0123 - val_loss: 0.0118
Epoch 9/10
1349/1349 [==============================] - 190s 141ms/step - loss: 0.0121 - val_loss: 0.0117
Epoch 10/10
1349/1349 [=======================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
